# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
# define necessary libraries

import pandas as pd
import torch
import json
import nltk
import random
import numpy as np
import keras
from collections import Counter
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'torch'

In [ ]:
# data reading

train_claims = json.load(open("data/train-claims.json", "r"))
dev_claims = json.load(open("data/dev-claims.json", "r"))
test_claims = json.load(open("data/test-claims-unlabelled.json", "r"))
evidences = json.load(open("data/evidence.json", "r"))

In [ ]:
# data processing

nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
nltk.download('stopwords')
stopwords_list = set(stopwords.words('english'))

def lemmatize_word(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma != word:
        return lemma
    return lemmatizer.lemmatize(word, 'n')

def lemmatize_remove_stop_words(text):
    words = text.lower().split()
    new_words = []
    for w in words:
        lemma = lemmatize_word(w)
        if lemma not in stopwords_list:
            new_words.append(lemma)
    return " ".join(new_words)


# Process evidences
evidences_texts = [lemmatize_remove_stop_words(evidence_text) for _, evidence_text in evidences.items()]
evidences_ids = list(evidences.keys())
evidences_id_dict = {evidence_id: idx for idx, evidence_id in enumerate(evidences_ids)}

# Process train set
train_ids = list(train_claims.keys())
train_texts = [lemmatize_remove_stop_words(data["claim_text"]) for _, data in train_claims.items()]
train_labels = [data["claim_label"] for _, data in train_claims.items()]
train_evidences = [[evidences_id_dict[i] for i in data["evidences"]] for _, data in train_claims.items()]

# Process dev set
dev_ids = list(dev_claims.keys())
dev_texts = [lemmatize_remove_stop_words(data["claim_text"]) for _, data in dev_claims.items()]
dev_labels = [data["claim_label"] for _, data in dev_claims.items()]
dev_evidences = [[evidences_id_dict[i] for i in data["evidences"]] for _, data in dev_claims.items()]

# Process test set
test_ids = list(test_claims.keys())
test_texts = [lemmatize_remove_stop_words(j["claim_text"]) for _, j in test_claims.items()]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(evidences_texts)

train_tfidf = vectorizer.transform(train_texts)
dev_tfidf = vectorizer.transform(dev_texts)
test_tfidf = vectorizer.transform(test_texts)
evidence_tfidf = vectorizer.transform(evidences_texts)

print(train_tfidf.shape)
print(evidence_tfidf.shape)

In [ ]:
train_cos_sims = np.dot(train_tfidf, evidence_tfidf.transpose()).toarray()
dev_cos_sims = np.dot(dev_tfidf, evidence_tfidf.transpose()).toarray()
test_cos_sims = np.dot(test_tfidf, evidence_tfidf.transpose()).toarray()

In [ ]:
def test_retrieval_topk(k, cur_scores, cur_labels):
    ACC = []
    top_ids = torch.topk(torch.FloatTensor(cur_scores), k, -1).indices.tolist()

    for i in range(len(cur_labels)):
        all_count = 0
        recall_count = 0
        for cur_ in cur_labels[i]:
            if cur_ in top_ids[i]:
                recall_count += 1
            all_count += 1
        ACC.append(recall_count / all_count)
    print(sum(ACC) / len(ACC))

topK = 10
test_retrieval_topk(topK, train_cos_sims, train_evidences)
test_retrieval_topk(topK, dev_cos_sims, dev_evidences)

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*